### 3. Scrape the list of third party drivers license locations from https://travel-id-documents.az.gov/authorized-third-party-driver-license-locations but include the link. Save as a CSV. Since it's more than just the text from the table, this requires actually using BeautifulSoup :(

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

url = 'https://travel-id-documents.az.gov/authorized-third-party-driver-license-locations'

# we're using some 'fuller' user-agent headers for this one, otherwise it doesn't really give us the table with our request.
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.5841.0 Safari/537.36'
}

response = requests.get(url, headers=headers)

print(response.status_code)

doc = BeautifulSoup(response.text)

200


<function print(*args, sep=' ', end='\n', file=None, flush=False)>

In [9]:
table = doc.find('table').find('tbody')
table

<tbody><tr><td><a href="http://az-mvd.com/" target="_blank">1 Stop Title &amp; Registration Services</a></td>
<td>940 N. Alma School Rd., #104<br/>
			Chandler, AZ 85224</td>
<td>480.821.3288</td>
<td>Mon.-Fri. 8:00 a.m.-6:00 p.m. Sat. 9:00 a.m.-4:30 p.m.</td>
</tr><tr><td><a href="http://az-mvd.com/" target="_blank">1 Stop Title &amp; Registration Services</a></td>
<td>5036 W. Cactus Rd., Ste. 2<br/>
			Glendale, AZ 85304</td>
<td>602.264.2400</td>
<td>Mon.-Fri. 8:00 a.m.-6:00 p.m. Sat. 8:30 a.m.-4:30 p.m.</td>
</tr><tr><td>Academy of Driving Motor Vehicle Center</td>
<td>4733 E. Broadway Blvd.<br/>
			Tucson, AZ 85711</td>
<td>520.750.7572</td>
<td>Mon.-Fri. 9 a.m.-5 p.m. and Sat. 9 a.m.-3 p.m.</td>
</tr><tr><td>Arizona Auto License</td>
<td>1337 W. Prince Rd<br/>
			Tucson, AZ 85705</td>
<td>520.696.2023</td>
<td>Driver License Hours: <br/>
			Mon.-Fri. 9 a.m.-5 p.m.<br/>
			Sat by Appt. Only</td>
</tr><tr><td>Arizona Auto License Service LLC</td>
<td>1457 N. Eliseo C Felix Jr.<br/>

In [11]:
table_rows = table.find_all('tr')
table_rows

[<tr><td><a href="http://az-mvd.com/" target="_blank">1 Stop Title &amp; Registration Services</a></td>
 <td>940 N. Alma School Rd., #104<br/>
 			Chandler, AZ 85224</td>
 <td>480.821.3288</td>
 <td>Mon.-Fri. 8:00 a.m.-6:00 p.m. Sat. 9:00 a.m.-4:30 p.m.</td>
 </tr>,
 <tr><td><a href="http://az-mvd.com/" target="_blank">1 Stop Title &amp; Registration Services</a></td>
 <td>5036 W. Cactus Rd., Ste. 2<br/>
 			Glendale, AZ 85304</td>
 <td>602.264.2400</td>
 <td>Mon.-Fri. 8:00 a.m.-6:00 p.m. Sat. 8:30 a.m.-4:30 p.m.</td>
 </tr>,
 <tr><td>Academy of Driving Motor Vehicle Center</td>
 <td>4733 E. Broadway Blvd.<br/>
 			Tucson, AZ 85711</td>
 <td>520.750.7572</td>
 <td>Mon.-Fri. 9 a.m.-5 p.m. and Sat. 9 a.m.-3 p.m.</td>
 </tr>,
 <tr><td>Arizona Auto License</td>
 <td>1337 W. Prince Rd<br/>
 			Tucson, AZ 85705</td>
 <td>520.696.2023</td>
 <td>Driver License Hours: <br/>
 			Mon.-Fri. 9 a.m.-5 p.m.<br/>
 			Sat by Appt. Only</td>
 </tr>,
 <tr><td>Arizona Auto License Service LLC</td>
 <td>14

In [17]:
# logic for extracting the text from each cell in the table
rows = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    try:
        # pulls out the link for its own column
        url = td[0].find('a')['href']
        row.append(url)
    except: pass
    rows.append(row)

rows[0:4]

[['1 Stop Title & Registration Services',
  '940 N. Alma School Rd., #104\n\t\t\tChandler, AZ 85224',
  '480.821.3288',
  'Mon.-Fri. 8:00 a.m.-6:00 p.m. Sat.\xa09:00 a.m.-4:30 p.m.',
  'http://az-mvd.com/'],
 ['1 Stop Title & Registration Services',
  '5036 W. Cactus Rd., Ste. 2\n\t\t\tGlendale, AZ 85304',
  '602.264.2400',
  'Mon.-Fri. 8:00 a.m.-6:00 p.m. Sat.\xa08:30 a.m.-4:30 p.m.',
  'http://az-mvd.com/'],
 ['Academy of Driving Motor Vehicle Center',
  '4733 E. Broadway Blvd.\n\t\t\tTucson, AZ 85711',
  '520.750.7572',
  'Mon.-Fri. 9 a.m.-5 p.m. and Sat. 9 a.m.-3 p.m.'],
 ['Arizona Auto License',
  '1337 W. Prince Rd\n\t\t\tTucson, AZ 85705',
  '520.696.2023',
  'Driver License Hours:\xa0\n\t\t\tMon.-Fri.\xa09 a.m.-5 p.m.\n\t\t\tSat by Appt. Only']]

In [20]:
df = []
df = pd.DataFrame(rows, columns=["Company","Address","Telephone","Hours","Link"])

In [22]:
# Should see the first five rows of the dataframe here
df.head()

,Company,Address,Telephone,Hours,Link
0,1 Stop Title & Registration Services,"940 N. Alma School Rd., #104\n\t\t\tChandler, ...",480.821.3288,Mon.-Fri. 8:00 a.m.-6:00 p.m. Sat. 9:00 a.m.-4...,http://az-mvd.com/
1,1 Stop Title & Registration Services,"5036 W. Cactus Rd., Ste. 2\n\t\t\tGlendale, AZ...",602.264.2400,Mon.-Fri. 8:00 a.m.-6:00 p.m. Sat. 8:30 a.m.-4...,http://az-mvd.com/
2,Academy of Driving Motor Vehicle Center,"4733 E. Broadway Blvd.\n\t\t\tTucson, AZ 85711",520.750.7572,Mon.-Fri. 9 a.m.-5 p.m. and Sat. 9 a.m.-3 p.m.,None
3,Arizona Auto License,"1337 W. Prince Rd\n\t\t\tTucson, AZ 85705",520.696.2023,Driver License Hours: \n\t\t\tMon.-Fri. 9 a.m....,None
4,Arizona Auto License Service LLC,"1457 N. Eliseo C Felix Jr.\n\t\t\tWay, Ste. 10...",623.925.5455 or Fax 623.925.5879,Mon.-Fri. 8 a.m.-5 p.m.,None


In [23]:
# Save to csv
df.to_csv("license-locations.csv")